# Actividad Reto: Simulación de Propagación (Fases 3 y 4)

Este notebook implementa la simulación de movimiento y contagio de personas en dos geometrías: **Ciudad Cuadrada** y **Ciudad Circular**. Se siguen los lineamientos de distribución uniforme inicial y movimiento aleatorio con distribución normal.

### Parámetros Generales
* **$N$:** Número total de personas.
* **$D$:** Dimensión del espacio (Lado o Diámetro).
* **$r$:** Radio de infección.
* **Estados:** 0 = Susceptible, 1 = Infectado, 2 = Recuperado.

---

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

# Configuración inicial para reproducibilidad
np.random.seed(42)

# --- PARÁMETROS DE LA SIMULACIÓN ---
N = 100              # Número de personas
D = 100              # Dimensión de la ciudad
r_infeccion = 8.0    # Radio de infección 'r'
tasa_recuperacion = 0.05 # Probabilidad de recuperación por iteración
n_iteraciones = 50   # Número de ciclos de tiempo
velocidad_std = 2.0  # Desviación estándar para el desplazamiento normal

def calcular_distancia(p1, p2, r):
    """
    Revisa la distancia euclidiana entre dos puntos.
    Regresa TRUE si la distancia es menor que r.
    """
    dist = np.linalg.norm(p1 - p2)
    return dist < r

## 1. Configuración Ciudad Cuadrada (Problema 1)

**Fase 3:**
* Se crea un área cuadrada de lado $D$.
* [cite_start]Las personas se distribuyen uniformemente usando `runif` (en Python `np.random.uniform`) en el rango $[0, D]$[cite: 8, 11].

**Fase 4:**
* [cite_start]En cada iteración, las personas se mueven una distancia determinada por una distribución normal[cite: 39].
* [cite_start]Si una persona susceptible está a una distancia menor a $r$ de un infectado, se infecta[cite: 43].
* [cite_start]Los infectados tienen una probabilidad de recuperarse.

In [3]:
def simulacion_cuadrada(N, D, r_inf, tasa_rec, n_steps):
    # --- FASE 3: INICIALIZACIÓN ---
    # [cite_start]Posiciones iniciales uniformes (0 a D) [cite: 8, 11]
    pos_x = np.random.uniform(0, D, N)
    pos_y = np.random.uniform(0, D, N)
    
    # [cite_start]Estado inicial: Todos susceptibles (0), al menos un infectado (1) [cite: 28, 29]
    estados = np.zeros(N, dtype=int) 
    estados[np.random.randint(0, N)] = 1 
    
    datos_simulacion = []

    # --- FASE 4: ITERACIÓN ---
    for t in range(n_steps): # Ciclo de iteraciones [cite: 37]
        
        # [cite_start]1. Guardar estado actual [cite: 45]
        for i in range(N):
            estado_label = "Susceptible"
            if estados[i] == 1:
                estado_label = "Infectado"
            elif estados[i] == 2:
                estado_label = "Recuperado"
            
            datos_simulacion.append({
                "iteracion": t,
                "id": i,
                "x": pos_x[i],
                "y": pos_y[i],
                "estado": estado_label
            })
        
        # [cite_start]2. Movimiento: Desplazamiento con distribución normal [cite: 39]
        dx = np.random.normal(0, velocidad_std, N)
        dy = np.random.normal(0, velocidad_std, N)
        
        # [cite_start]Sumar desplazamiento a la posición [cite: 40]
        pos_x += dx
        pos_y += dy
        
        # Mantener dentro de los límites (Ciudad Cuadrada)
        pos_x = np.clip(pos_x, 0, D)
        pos_y = np.clip(pos_y, 0, D)
        
        # [cite_start]3. Actualización de estados (Infección y Recuperación) [cite: 41]
        nuevos_infectados = []
        indices_infectados = np.where(estados == 1)[0]
        indices_susceptibles = np.where(estados == 0)[0]
        
        # [cite_start]Lógica de infección: Distancia < r [cite: 42, 43]
        for s_idx in indices_susceptibles:
            p_s = np.array([pos_x[s_idx], pos_y[s_idx]])
            for i_idx in indices_infectados:
                p_i = np.array([pos_x[i_idx], pos_y[i_idx]])
                if calcular_distancia(p_s, p_i, r_inf):
                    nuevos_infectados.append(s_idx)
                    break 
        
        if len(nuevos_infectados) > 0:
            estados[nuevos_infectados] = 1
        
        # [cite_start]Lógica de recuperación: Aleatorio < tasa [cite: 44]
        for i_idx in indices_infectados:
            if np.random.uniform(0, 1) < tasa_rec:
                estados[i_idx] = 2 

    return pd.DataFrame(datos_simulacion)

# Ejecutar y visualizar
df_sq = simulacion_cuadrada(N, D, r_infeccion, tasa_recuperacion, n_iteraciones)

# [cite_start]Animación con Plotly [cite: 46]
fig = px.scatter(df_sq, x="x", y="y", animation_frame="iteracion", animation_group="id",
           color="estado", hover_name="id", range_x=[0,D], range_y=[0,D],
           color_discrete_map={"Susceptible": "blue", "Infectado": "red", "Recuperado": "green"},
           title="Simulación Ciudad Cuadrada")
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig.show()

## 2. Configuración Ciudad Circular (Problema 2)

**Fase 3:**
* [cite_start]Se crea una ciudad circular de radio $R = D/2$[cite: 13].
* [cite_start]**Distribución:** Para distribuir uniformemente en un círculo, el radio se calcula como $R \sqrt{u}$ donde $u$ es aleatorio uniforme[cite: 13, 24].

**Fase 4:**
* Misma lógica de movimiento e infección, pero verificando que las personas se mantengan dentro del radio del círculo.

In [4]:
def simulacion_circular(N, D, r_inf, tasa_rec, n_steps):
    radio_ciudad = D / 2
    
    # --- FASE 3: INICIALIZACIÓN CIRCULAR ---
    # [cite_start]Distribución uniforme modificada para círculo [cite: 13, 24]
    theta = np.random.uniform(0, 2 * np.pi, N)
    u = np.random.uniform(0, 1, N)
    r_pos = radio_ciudad * np.sqrt(u) # Raíz cuadrada para uniformidad
    
    # Coordenadas polares a cartesianas
    pos_x = r_pos * np.cos(theta)
    pos_y = r_pos * np.sin(theta)
    
    estados = np.zeros(N, dtype=int)
    estados[np.random.randint(0, N)] = 1
    
    datos_simulacion = []

    # --- FASE 4: ITERACIÓN ---
    for t in range(n_steps):
        
        # Guardar datos
        for i in range(N):
            estado_label = "Susceptible"
            if estados[i] == 1: estado_label = "Infectado"
            elif estados[i] == 2: estado_label = "Recuperado"
            
            datos_simulacion.append({
                "iteracion": t, "id": i, "x": pos_x[i], "y": pos_y[i], "estado": estado_label
            })
        
        # [cite_start]Movimiento Normal [cite: 39]
        dx = np.random.normal(0, velocidad_std, N)
        dy = np.random.normal(0, velocidad_std, N)
        
        # Calcular nueva posición tentativa
        nuevo_x = pos_x + dx
        nuevo_y = pos_y + dy
        
        # Validar límites circulares (si sale, se queda en posición anterior o se ajusta)
        dist_centro = np.sqrt(nuevo_x**2 + nuevo_y**2)
        mask_dentro = dist_centro <= radio_ciudad
        
        # Solo actualizamos si están dentro del radio
        pos_x = np.where(mask_dentro, nuevo_x, pos_x)
        pos_y = np.where(mask_dentro, nuevo_y, pos_y)
        
        # [cite_start]Infección y Recuperación (Idéntico a lógica anterior) [cite: 42, 44]
        nuevos_infectados = []
        indices_infectados = np.where(estados == 1)[0]
        indices_susceptibles = np.where(estados == 0)[0]
        
        for s_idx in indices_susceptibles:
            p_s = np.array([pos_x[s_idx], pos_y[s_idx]])
            for i_idx in indices_infectados:
                p_i = np.array([pos_x[i_idx], pos_y[i_idx]])
                if calcular_distancia(p_s, p_i, r_inf):
                    nuevos_infectados.append(s_idx)
                    break 
        
        estados[nuevos_infectados] = 1
        
        for i_idx in indices_infectados:
            if np.random.uniform(0, 1) < tasa_rec:
                estados[i_idx] = 2

    return pd.DataFrame(datos_simulacion)

# Ejecutar
df_circ = simulacion_circular(N, D, r_infeccion, tasa_recuperacion, n_iteraciones)

# [cite_start]Animación Circular [cite: 46]
fig_c = px.scatter(df_circ, x="x", y="y", animation_frame="iteracion", animation_group="id",
           color="estado", hover_name="id", range_x=[-D/1.5, D/1.5], range_y=[-D/1.5, D/1.5],
           color_discrete_map={"Susceptible": "blue", "Infectado": "red", "Recuperado": "green"},
           title="Simulación Ciudad Circular")

# Dibujar borde
fig_c.add_shape(type="circle", xref="x", yref="y", x0=-D/2, y0=-D/2, x1=D/2, y1=D/2, line_color="Black")
fig_c.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 100
fig_c.update_layout(width=600, height=600)
fig_c.show()